In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import preprocessor as p

In [4]:
df = pd.read_csv('tweets_stocks_combined_5mins.csv')

In [59]:
X = df.loc[:, 'cleaned_text']
y = df.loc[:, '5mins_price_diff_perc']

In [62]:
!pip install tweet-preprocessor

def clean_text(text):
    '''
    lowercase, remove puncs, remove urls
    '''
    text = p.clean(text)
    text = text.lower()
    #text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r"http\S+", ' ', text)
    return text
    
clean = lambda x: clean_text(x)

def df_to_xy(df):
    x = df.text.apply(clean)
    y = df.PP.apply(lambda xx: xx[75])
    #y = y.values
    return (x,y)

X = X.apply(clean)

AttributeError: module 'preprocessor' has no attribute 'clean'

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
X_train

1702    Kim Jong Un of North Korea proclaims “unwaveri...
2164    Prime Minister Trudeau is being so indignant, ...
1281    “Trump gets no credit for what he’s done in th...
1866    ....China, which is for the first time doing p...
306     Two dozen NFL players continue to kneel during...
                              ...                        
1638    Best economic numbers in decades. If the Democ...
1095    "Is it legal for a sitting President to be ""w...
1130    The FAKE NEWS media (failing @nytimes, @CNN, @...
1294    As I predicted all along, Obamacare has been s...
860     KAREN HANDEL FOR CONGRESS. She will fight for ...
Name: cleaned_text, Length: 1788, dtype: object

In [28]:
corpus_list = []

for i in X_train:
    corpus_list.append(i.split(' '))

In [53]:
word2vec_model = Word2Vec(corpus_list, min_count=1, size=100)
print(word2vec_model)

Word2Vec(vocab=10608, size=100, alpha=0.025)


In [63]:
print(word2vec_model.syn0)

AttributeError: 'Word2Vec' object has no attribute 'syn0'

In [54]:
words = list(word2vec_model.wv.vocab)
print(words)

['Kim', 'Jong', 'Un', 'of', 'North', 'Korea', 'proclaims', '“unwavering', 'faith', 'in', 'President', 'Trump.”', 'Thank', 'you', 'to', 'Chairman', 'Kim.', 'We', 'will', 'get', 'it', 'done', 'together!', 'Prime', 'Minister', 'Trudeau', 'is', 'being', 'so', 'indignant,', 'bringing', 'up', 'the', 'relationship', 'that', 'U.S.', 'and', 'Canada', 'had', 'over', 'many', 'years', 'all', 'sorts', 'other', 'things...but', 'he', 'doesn’t', 'bring', 'fact', 'they', 'charge', 'us', '300%', 'on', 'dairy', '—', 'hurting', 'our', 'Farmers,', 'killing', 'Agriculture!', '“Trump', 'gets', 'no', 'credit', 'for', 'what', 'he’s', 'Middle', 'East.”', '', '@IngrahamAngle', 'So', 'true,', 'thank', 'Laura!', '....China,', 'which', 'first', 'time', 'doing', 'poorly', 'against', 'us,', 'spending', 'a', 'fortune', 'ads', 'P.R.', 'trying', 'convince', 'scare', 'politicians', 'fight', 'me', 'Tariffs-', 'because', 'are', 'really', 'their', 'economy.', 'Likewise', 'countries.', 'Winning,', 'but', 'must', 'be', 'stron

In [55]:
print(word2vec_model['Barack'].shape)

(100,)


C:\Users\brgoh\anaconda33\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
def pretrained_embedding_layer(word2vec_model, vocab_len):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_model -- dictionary mapping words to their word2vec vector representations
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = vocab_len + 1 # adding 1 to fit Keras embedding (requirement)
    emb_dim = 50
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    
    
    
    
    
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        if word_to_vec_map[word].shape[0] == emb_dim:
            emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it non-trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = keras.layers.Embedding(input_dim=vocab_len, output_dim=emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer